# Ranking biscuits on a supermarket website using a Bayesian approach

**Problem statement: what is the most effective order to display different types of biscuits on a supermarket website?**

It really depends on our objective: is it conversion? Profitability? Customer feedback? Or other metrics? In this notebook, I am going to demonstrate an ensemble Bayesian approach to rank biscuits by their weighted scores of expected profit and star rating.

Many sorting algorithm suffers badly from simply taking averages of metrics like conversion and ignoring uncertainties. For example, a type of biscuits with 9 purchases from 100 visits is *more likely* to be better than another type of biscuits with 1 purchase from 10 visits. However simply calculating the averages will give you the opposite.

This happens because using an average to estimate the expected number **only** works if the number of samples is large (ideally infinite).

In this notebook, I am going to tackle this problem by ranking biscuits using the **95% least plausible value** which means that the true parameter has only 5% chance being below this value. This idea is to incorporate uncertainty in the ranking procedure.

In [1]:
import numpy as np
from scipy.stats import norm, uniform, gamma, lognorm, zscore, chi2
import pymc3 as pm
import pandas as pd
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')
figsize(14,5)

# Simulate data

This is a simulation of aggregated data over a period of time.

In [2]:
def get_randint(low, high, size, seed=0):
    np.random.seed(seed)
    return np.random.randint(low, high, size)
    

data = pd.DataFrame(
{'biscuits' : [chr(letter).upper() for letter in range(97, 123)],
 'num_visit'  : get_randint(low=0, high=50000, size=26, seed=0),
 'num_purchase'  : list(map(int, chi2.rvs(10, size=26, random_state=0))),
 'star_1': get_randint(low=0, high=10, size=26, seed=1),
 'star_2': get_randint(low=0, high=10, size=26, seed=2),
 'star_3': get_randint(low=0, high=10, size=26, seed=3),
 'star_4': get_randint(low=0, high=10, size=26, seed=4),
 'star_5': get_randint(low=0, high=10, size=26, seed=5),
 'profit': chi2.rvs(3, size=26, random_state=1)
})

data.head()

,biscuits,num_visit,num_purchase,star_1,star_2,star_3,star_4,star_5,profit
0,A,2732,19,5,8,8,7,3,7.895238
1,B,43567,11,8,8,9,5,6,1.245585
2,C,42613,19,9,6,3,1,6,1.368224
3,D,45891,5,5,2,8,8,0,0.698250
4,E,21243,8,0,8,8,7,9,8.496419


# Ensemble model

I will first calculate scores for conversion which will be used to calculate the expected profit, then separately scores for star rating. Finally calculate the weighted sum of those scores to obtain the final scores for ranking.

In the following sections, I will infer a distribution of each parameters of interest using Bayesian methods (rather than simply looking at the point estimates).

## 1. Conversion

### Infer conversion distribution of each biscuits option

- To express my ignorance to the prior distribution of conversion, I use $Beta(1, 1)$, i.e. a uniform distribution from 0 to 1 as my prior distribution.

- The evidence/likelihood is a sequence of N independent trials, so it is modelled by a Binomial distribution $B(n=N, p=suc/N)$.

- Since Beta and binomial distributions have a conjugate relationship, the posterior distribution has a closed form solution (MCMC is not required here):  

$$Beta(a, b)$$ 
where $$a = 1 + suc$$ and $$b = 1 + (N - suc)$$

- This posterior distribution represents a distribution of a biscuits option's conversion.

### Expected profit samples

Expected profit samples can be obtained from conversion samples, multiplied by profit of selling a pack of biscuits.

### Evaluate the 95% least plausible value

Find out the 95% least plausible value by ordering the expected profit array and select the (1 - 95%)th item

In [3]:
def get_exp_profit_lpv(N, suc, pf, threshold=0.95):
    """
    Expected profit least plausible value (lpv)
    
    Parameters
    ---
    N: int
        Total number of trials (look)
    suc: int
        Number of successes (book)
    pf: float
        Profit of selling a pack of biscuits
    threshold: float
        
    """
    
    # params for conversion posterior distribution
    a = 1 + suc
    b = 1 + (N - suc)
    
    # get posterior samples
    size = 20000
    posterior_samples = np.random.beta(a, b, size=size)
    
    # expected profit = conversion x profit of a pack of biscuits
    exp_profit_samples = posterior_samples * pf
    
    # least plausible value
    lpv = np.sort(exp_profit_samples)[int((1 - threshold) * size)]
    
    return lpv

In [4]:
data['profit_score'] = data.apply(lambda x: get_exp_profit_lpv(x.num_visit, x.num_purchase, x.profit), axis=1)

In [5]:
data.head()

,biscuits,num_visit,num_purchase,star_1,star_2,star_3,star_4,star_5,profit,profit_score
0,A,2732,19,5,8,8,7,3,7.895238,0.038265
1,B,43567,11,8,8,9,5,6,1.245585,0.000198
2,C,42613,19,9,6,3,1,6,1.368224,0.000425
3,D,45891,5,5,2,8,8,0,0.698250,0.000041
4,E,21243,8,0,8,8,7,9,8.496419,0.001872


## 2. 5-star rating scheme

- This is a similar problem to conversion, but here there are multiple options (1-5 stars) rather than boolean (book or not book).

- Bayesian estimation works well in this case because some biscuits only have very small amount of ratings, for example a new brand. 

- Instead of focusing on the probability of each option being selected, we will evaluate the **expected star rating** and its uncertainty of each biscuits option. The expected value can be calculated using:

$$E[S] = 0p_0 + 1p_1 + 2p_2 + 3p_3 + 4p_4 + 5p_5$$

Where $p_n$ is the probability of a rating being selected and 

$$\sum_{i=0}^{5} p_n = 1$$

- Note that $p_0$ is the probability of no rating being given.

### Infer star rating distribution of each biscuit option

- A 5-star rating scheme can be expressed as a Multinomial distribution where each rating has its own probability of being selected:

$$Multinomial(N_0, N_1, N_2, N_3, N_4, N_5)$$ 

Where $N_m$ is the number of observations of a star rating

- To express my ignorance to the prior distribution we will use a Dirichlet distribution which is a multivariate generalisation of a Beta distribution:

$$Dirichlet(1, 1, 1, 1, 1, 1)$$

- Conveniently, a Dirichlet distribution is a conjugate prior to a Multinomial distribution (hence MCMC is not required). The posterior distribution is:

$$Dirichlet(a_0 + N_0, a_1 + N_1, ..., a_5 + N_5)$$

Where $a_m$ are the parameters in the prior Dirichlet distribtion (all of them are 1 in this case)

- $N_0$ is the number of visitors who did not give a rating. It is difficult to obtain this number in this case as we do not know the total number of visitors who are 'eligible' to rate (for example buyers who have purchased and tasted the biscuits, not potential buyers). But we do not need to know as it will be multiplied by 0 in the expectation equation, and in the Dirichlet posterior calculation, the observations are assumed to be independent - removing $p_0$ will not affect the calculation of the other variables ($p_1, p_2, ...$).

- To obtain a distribution of the expected rating, we sample from the posterior Dirichlet distribution and then multiply with an array of *rewards*

- Finally we can find out the 95% least plausible value by ordering the expected rating array and select the (1 - 95%)th item

In [6]:
def get_star_lpv(N, R, threshold=0.95):
    """
    Star rating: least plausible value (lpv)
    
    Parameters
    ---
    N: list
        List of occurrences of each star rating option
    R: list
        List of rewards associated to each star rating option
    threshold: float
        Least plausible value
    """
    
    # Calculate posterior params
    observations = np.array(N)
    prior_params = np.array(np.ones_like(N))
    post_params = prior_params + observations
    
    # Obtain posterior samples
    size=20000
    posterior_samples = np.random.dirichlet(post_params, size=size)
    
    # Samples of expected values
    exp_samples = np.sum(np.multiply(posterior_samples, R), axis=1)
    
    # Least plausible value
    lpv = np.sort(exp_samples)[int((1 - threshold) * size)]
    
    return lpv

In [7]:
rate_rewards = [1, 2, 3, 4, 5]
data['star_score'] = \
    data.apply(lambda x: get_star_lpv([x.star_1, x.star_2, x.star_3, x.star_4, x.star_5], rate_rewards), axis=1)

In [8]:
data.head()

,biscuits,num_visit,num_purchase,star_1,star_2,star_3,star_4,star_5,profit,profit_score,star_score
0,A,2732,19,5,8,8,7,3,7.895238,0.038265,2.533002
1,B,43567,11,8,8,9,5,6,1.245585,0.000198,2.481128
2,C,42613,19,9,6,3,1,6,1.368224,0.000425,2.187215
3,D,45891,5,5,2,8,8,0,0.698250,0.000041,2.482245
4,E,21243,8,0,8,8,7,9,8.496419,0.001872,3.137841


## 3. Ensemble: weighted sum of all scores

- In the last step we need to combine the scores from profit and star rating to rank the biscuits options


- Since the scores from each metric is on a different scale, we need to first standardise those scores. It is also known as the 'z-score'. This centres the values of each metric at zero and scales the distribution with units of standard deviation


- To further customise the ranking, we can assign weights to each metric to obtain a weighted sum of those scores. For instance, if we deem profit more important we can put a higher weight to it: (profit: 2, star_rating: 1)


- Finally we sort the biscuits options by the weighted score

In [9]:
def get_weighted_sum(raw_scores, weights):
    std = zscore(raw_scores, axis=1)
    weighted = np.multiply(std.T, weights)
    weighted_sum = weighted.sum(axis=1)
    return weighted_sum

In [10]:
score_cols = ['profit_score', 'star_score']
weights = np.array([2, 1])
raw_scores = np.array([data[c].values for c in score_cols])
data['weighted_score'] = get_weighted_sum(raw_scores, weights)

In [11]:
data.sort_values('weighted_score', ascending=False)

,biscuits,num_visit,num_purchase,star_1,star_2,star_3,star_4,star_5,profit,profit_score,star_score,weighted_score
0,A,2732,19,5,8,8,7,3,7.895238,0.038265,2.533002,9.628646
4,E,21243,8,0,8,8,7,9,8.496419,0.001872,3.137841,1.342468
20,U,35665,15,1,3,2,5,9,0.656266,0.000184,3.267938,1.246444
5,F,30403,11,0,7,0,8,8,1.044981,0.000239,3.206327,1.087315
22,W,27469,5,0,8,3,9,9,1.942971,0.000184,3.203896,1.065350
10,K,14935,17,2,4,5,7,7,1.980315,0.001536,3.048019,0.996522
13,N,39512,12,2,3,0,4,7,5.782686,0.001133,2.978420,0.689241
6,G,32103,13,1,2,5,2,4,7.131374,0.001872,2.826409,0.462876
21,V,16921,15,7,5,1,5,9,2.877246,0.001702,2.660737,-0.051699
7,H,41993,9,7,1,3,9,7,0.112744,0.000014,2.797300,-0.129494


In [12]:
# data.to_csv('rank_results.csv', index=False)

# Reference

- https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers
- https://fulmicoton.com/posts/bayesian_rating/